### initialization

In [ ]:
from pathlib import Path
SRC_PATH = Path('/home/ec2-user/SageMaker/david/tdm-sentiment/src/')
import sys
sys.path.append(str(SRC_PATH))
from config import *
import pandas as pd
import logging

is_economic_model = is_economic_module.EconomicClassifier(model_path=IS_ECONOMIC_MODEL, initialize=True)  # Initialize the economic model
tdm_parser = tdm_parser_module.TdmXmlParser()

corpuses_dir = CORPUSES_PATH  #'all_dataset_file_names.txt' LosAngelesTimesDavid all_dataset_file_names.txt # Path to the input
output_path = FILE_NAMES_PATH / 'is_economic_train_files.txt'  # Path to the output text file
csv_path = PROJECT_DATA_PATH / 'train_data' / 'economic_classifier_train_data.csv'
file_names_path = FILE_NAMES_PATH / 'is_economic_train_files.txt'  # 'all_dataset_file_names.txt'  # Path to the file containing names


# set logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.handlers.clear()

handler = logging.FileHandler(filename=LOGS_PATH / 'train_is_economic_info.log') 
formatter =logging.Formatter('%(asctime)s - %(name)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

### train model 

In [ ]:
# try get data training dataset if exists
try:
    df = pd.read_csv(csv_path)
    print(df.head(2))
except:    
    # get sample file
    file_process.get_file_paths_sample(corpuses_dir, output_path, sample_size=500*10**3)

    # crate and save df
    df = file_process.load_df_from_xml(corpuses_dir, file_names_path)
    # save df
    csv_path.parent.mkdir(parents=True, exist_ok=True) # Ensure the output directory exists
    df.to_csv(csv_path, index=False)
    logger.info(f"Training data saved to {csv_path}")
    #print(df.head(2))

# Assuming df is a DataFrame with the necessary columns
X_test, y_test = is_economic_model.train_classifier(df, data_column='paragrph_text', label_column='section')
# is_economic_model.save_model() # be careful to make a copy of current one first 

logger.info(f'model evaluation with thrashold=0.5 is: \n {is_economic_model.evaluate_model(X_test, y_test, threshold=0.5)}')
logger.info(f'model evaluation with thrashold=0.7 is: \n {is_economic_model.evaluate_model(X_test, y_test, threshold=0.7)}')